# Using Plotly

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from matplotlib.patches import FancyBboxPatch
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea
import seaborn as sns
import pprint
from sodapy import Socrata
import os
import plotly.graph_objects as go


In [ ]:
# Replace None with your app token "AVDM_Competencies" for authenticated access
client = Socrata("analisi.transparenciacatalunya.cat", "LvvKUfbMJLH9IdgS7C4hXbeye")

# Fetch the first 2000 results as JSON and convert them to a Python list of dictionaries
results = client.get("rk5x-gny6",limit=1000000)

# Convert to pandas DataFrame
comp = pd.DataFrame.from_records(results)

# Modify what it is wrong in the dataframe:
comp['genere'] = comp['genere'].replace({'Home': 'H', 'Dona': 'D'})

# Calculate mean grades, not french
comp[['pcat', 'pmat', 'pcast', 'pmed', 'pang']] = comp[['pcat', 'pmat', 'pcast', 'pmed', 'pang']].apply(pd.to_numeric, errors='coerce')
comp['mean_grade'] = comp[['pcat', 'pmat', 'pcast', 'pmed', 'pang']].mean(axis=1)

In [9]:
c_priv = comp[comp["naturalesa"] == "Privat"].copy()
c_priv.rename(columns={'any': 'curs_escolar'}, inplace=True)

c_pub = comp[comp["naturalesa"] == "Públic"].copy()
c_pub.rename(columns={'any': 'curs_escolar'}, inplace=True)

In [11]:
# Data cleaning (replace with actual data)
year = '2023'
pub23 = c_pub[c_pub['curs_escolar'] == year].copy()
priv23 = c_priv[c_priv['curs_escolar'] == year].copy()

pub23_clean = pd.to_numeric(pub23['mean_grade'], errors='coerce').dropna()
priv23_clean = pd.to_numeric(priv23['mean_grade'], errors='coerce').dropna()

# Compute histograms
hist1, bin_edges1 = np.histogram(pub23_clean, bins=30, density=True)
bin_centers1 = (bin_edges1[1:] + bin_edges1[:-1]) / 2

hist2, bin_edges2 = np.histogram(priv23_clean, bins=30, density=True)
bin_centers2 = (bin_edges2[1:] + bin_edges2[:-1]) / 2

# Create the Plotly figure
fig = go.Figure()

# Add Public histogram
fig.add_trace(go.Bar(
    x=bin_centers1,
    y=hist1,
    name='Public',
    marker=dict(color='skyblue'),
    hoverinfo='x+y'
))

# Add Private histogram
fig.add_trace(go.Bar(
    x=bin_centers2,
    y=hist2,
    name='Private',
    marker=dict(color='salmon'),
    hoverinfo='x+y'
))

# Customize the layout
fig.update_layout(
    title="Histograms of Competencies' mean grades in 2023",
    xaxis_title="Mean grade",
    yaxis_title="Density",
    barmode='overlay',
    legend_title="School Type",
    template='plotly_white',
    hovermode='x'
)

# Show the plot
fig.show()
